# Генерация датафрейма для обучения общей модели

## Загрузим и импортируем нужные модули


In [1]:
!pip install -q catboost

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


## Выгрузим наши датасеты

In [5]:
SEED = 42
PATH = '/kaggle/input/tractor-data/'

In [6]:
data_anomaly = pd.read_csv(PATH + "dataset._anomaly.csv", sep=";", decimal = ',')
data_normal = pd.read_csv(PATH + "dataset._normal.csv", sep=";", decimal = ',')
data_problems = pd.read_csv(PATH + "dataset._problems.csv", sep=";", decimal = ',')

/tmp/ipykernel_34/652216023.py:2: DtypeWarning: Columns (1,3,5,12,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  data_normal = pd.read_csv(PATH + "dataset._normal.csv", sep=";", decimal = ',')
/tmp/ipykernel_34/652216023.py:3: DtypeWarning: Columns (1,3,5,12,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  data_problems = pd.read_csv(PATH + "dataset._problems.csv", sep=";", decimal = ',')


## Произведем препроцессинг данных

In [7]:
features = [col for col in data_problems.columns if col not in ['Дата и время']]

In [8]:
data_normal.replace("-",np.nan,inplace=True)
data_normal.replace('        -',np.nan,inplace=True)
data_normal = data_normal[features].dropna(how='all')

data_problems.replace("-",np.nan,inplace=True)
data_problems.replace('        -',np.nan,inplace=True)
data_problems = data_problems[features].dropna(how='all')

data_anomaly.replace("-",np.nan,inplace=True)
data_anomaly.replace('        -',np.nan,inplace=True)
data_anomaly = data_anomaly[features].dropna(how='all')

data_normal['problem'] = np.array([0]*len(data_normal))
data_problems['problem'] = np.array([1]*len(data_problems))
data_anomaly['problem'] = np.array([0]*len(data_anomaly))

data_normal['anomaly'] = np.array([0]*len(data_normal))
data_problems['anomaly'] = np.array([0]*len(data_problems))
data_anomaly['anomaly'] = np.array([1]*len(data_anomaly))

/tmp/ipykernel_34/2537589906.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_normal.replace("-",np.nan,inplace=True)
/tmp/ipykernel_34/2537589906.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_normal.replace('        -',np.nan,inplace=True)
/tmp/ipykernel_34/2537589906.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('fut

In [9]:
metki = ['Нейтраль КПП (spn3843)', 'Стояночный тормоз (spn3842)',
       'Аварийная температура охлаждающей жидкости (spn3841)',
       'Засоренность воздушного фильтра (spn3840)',
       'Засоренность фильтра КПП (spn3847)',
       'Аварийное давление масла ДВС (spn3846)',
       'Засоренность фильтра ДВС (spn3845)',
       'Засоренность фильтра рулевого управления (spn3844)',
       'Засоренность фильтра навесного оборудования (spn3851)',
       'Недопустимый уровень масла в гидробаке (spn3850)',
       'Аварийная температура масла в гидросистеме (spn3849)',
       'Аварийное давление в I контуре тормозной системы (spn3848)',
       'Аварийное давление в II контуре тормозной системы (spn3855)',
       'Зарядка АКБ (spn3854)', 'Отопитель (spn3853)',
       'Выход блока управления двигателем (spn3852)',
       'Включение тормозков (spn3859)', 'Засоренность фильтра слива (spn3858)',
       'Аварийное давление масла КПП (spn3857)',
       'Аварийная температура масла ДВС(spn3856)',
       'Неисправность тормозной системы (spn3863)', 'Термостарт (spn3862)',
       'Разрешение запуска двигателя (spn3861)', 'Низкий уровень ОЖ (spn3860)',
       'Аварийная температура масла ГТР (spn3867)',
       'Необходимость сервисного обслуживания (spn3866)',
       'Подогрев топливного фильтра (spn3865)', 'Вода в топливе (spn3864)',
       'Холодный старт (spn3871)', 'Крутящий момент (spn513), Нм',
       'Положение рейки ТНВД (spn51), %', 'Расход топлива (spn183), л/ч',
       'ДВС. Температура наддувочного воздуха, °С',
       'Давление наддувочного воздуха двигателя (spn106), кПа',
       'Текущая передача (spn523)',
       'Температура масла гидравлики (spn5536), С', 'Педаль слива (spn598)']

In [10]:
def to_standart(x):
    try:
        return int(x)
    except:
        return 0

data_anomaly[metki] = data_anomaly[metki].map(lambda x: to_standart(x))
data_problems[metki] = data_problems[metki].map(lambda x: to_standart(x))
data_normal[metki] = data_normal[metki].map(lambda x: to_standart(x))

In [11]:
def to_time(x):
    try:
        return int(x[:3])*60+int(x[-2:])
    except:
        return np.nan
data_anomaly['Значение счетчика моточасов, час:мин'] = data_anomaly['Значение счетчика моточасов, час:мин'].map(lambda x: to_time(x))
data_problems['Значение счетчика моточасов, час:мин'] = data_problems['Значение счетчика моточасов, час:мин'].map(lambda x: to_time(x))
data_normal['Значение счетчика моточасов, час:мин'] = data_normal['Значение счетчика моточасов, час:мин'].map(lambda x: to_time(x))

In [12]:
num_feat = ['Полож.пед.акселер.,%', 'Нагрузка на двигатель, %',
       'Давл.масла двиг.,кПа', 'Темп.масла двиг.,°С', 'Обор.двиг.,об/мин',
       'Значение счетчика моточасов, час:мин', 
       'КПП. Температура масла', 'КПП. Давление масла в системе смазки',
       'Скорость', 'ДВС. Давление смазки',
       'ДВС. Температура охлаждающей жидкости',
       'Давление в пневмостистеме (spn46), кПа', 'Уровень топлива % (spn96)',
       'Электросистема. Напряжение', 'ДВС. Частота вращения коленчатого вала']
cat_feat = ['iButton2','Сост.пед.сцепл.']
features = num_feat+cat_feat
len(features)

17

In [13]:
data_anomaly[cat_feat] = data_anomaly[cat_feat].fillna('no_data')
data_problems[cat_feat] = data_problems[cat_feat].fillna('no_data')
data_normal[cat_feat] = data_normal[cat_feat].fillna('no_data')
data_anomaly[cat_feat] = data_anomaly[cat_feat].astype("category")
data_problems[cat_feat] = data_problems[cat_feat].astype("category")
data_normal[cat_feat] = data_normal[cat_feat].astype("category")

## Создание нового датасета (включающего все данные со всеми метками)

In [14]:
df_all = pd.concat([data_normal, data_anomaly, data_])

In [15]:
for_rep = []
for feat in num_feat:
    try:
        df_all[feat] = df_all[feat].astype(float)
    except:
        for_rep.append(feat)

In [16]:
def to_int(x):
    return str(x).replace(',','.')
df_all[for_rep] = df_all[for_rep].map(lambda x: to_int(x))

In [17]:
df_anomaly[num_feat] = df_anomaly[num_feat].astype("float64")

In [ ]:
df_all.to_csv('df_all.csv')